In [ ]:
# 분석대상 이름 입력
name = input("분석대상 이름을 입력하세요.")
max_freq = int(input("분석할 최대 주파수를 입력하시오")) + 1
bad_channels_name = input("제거할 채널 이름을 입력하세요. 없으면 enter를 누르세요.")

# TFR 시행
print(f"{name}의 TFR을 시행하겠습니다.")
# fif 파일 불러오기
file_path = rf'H:\Mg_EEG\edf_files\{name}_7200_clean.fif'
clean_data = mne.io.read_raw_fif(file_path, preload=True)
print(f"{name}의 fif 파일을 성공적으로 불러왔습니다: {file_path}")
    
# Delete data for a specific channel
if bad_channels_name:
    bad_channels_list = [channel.strip() for channel in bad_channels_name.split(',')]
    clean_data.drop_channels(bad_channels_list)
    
# Computing TFR
print(f"{max_freq}Hz까지의 TFR을 계산 중 입니다.")
tfr = clean_data.compute_tfr(method='multitaper', freqs=np.arange(1, max_freq), tmin=0, tmax=7199, n_jobs=-1, reject_by_annotation=False)
print(f"{name}의 TFR 계산이 완료되었습니다.")

In [ ]:
# 상황에 맞게 이름 및 경로 설정하여 사용하기

# Saving TFR file
saving_path = rf'H:\Mg_EEG\tfr_files\{name}_7200_tfr.h5'
tfr.save(saving_path, overwrite=False)
print(f"TFR 파일이 성공적으로 저장되었습니다.")

# # Saving TFR file
# saving_path = rf'H:\Mg_EEG\tfr_files\{name}gamma_7200_tfr.h5'
# tfr.save(saving_path, overwrite=False)
# print(f"TFR 파일이 성공적으로 저장되었습니다.")